**Connect google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Aug 14 23:10:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 0.PW6168
'1-Big picture.gdoc'
 1.Deere
 4.Edu
 AIFull.pdf
'AI-zero2pro (x2x).gdoc'
 Colab
 Draw.io.me
'Qt memo.gdoc'
 README.md
'Student Files'
 tmp
'Using NASA Satellites to Monitor a Changing Earth.pdf'
 yolov3


**1) Clone, configure & compile Darknet**



In [4]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14240, done.
remote: Total 14240 (delta 0), reused 0 (delta 0), pack-reused 14240
Receiving objects: 100% (14240/14240), 12.79 MiB | 9.75 MiB/s, done.
Resolving deltas: 100% (9677/9677), done.


In [5]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [6]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [7]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# # Change lines in yolov3.cfg file: 3 classes
# !sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
# !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# !sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
# !sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
# !sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
# !sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
# !sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [8]:
# Change lines in yolov3.cfg file: 2 classes
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [9]:
!mkdir -p /mydrive/Colab/CV_Bee/Models

In [11]:
!echo -e 'b\np' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Colab/CV_Bee/Models' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [13]:
!mkdir -p /mydrive/Colab/CV_Bee/Configs
!cp cfg/yolov3_training.cfg /mydrive/Colab/CV_Bee/Configs/yolov3_testing.cfg
!cp data/obj.names /mydrive/Colab/CV_Bee/Configs/classes.txt

**5) Create a folder and unzip image dataset**

In [15]:
!mkdir -p data/obj
!rm data/obj/* -r
!unzip "/mydrive/Colab/CV_Bee/Dataset_for_yolo3.zip" -d data/obj

Archive:  /mydrive/Colab/CV_Bee/Dataset_for_yolo3.zip
   creating: data/obj/Dataset/
  inflating: data/obj/Dataset/NP10057-126r.jpg  
 extracting: data/obj/Dataset/NP10057-126r.txt  
  inflating: data/obj/Dataset/NP10109-128r.jpg  
 extracting: data/obj/Dataset/NP10109-128r.txt  
  inflating: data/obj/Dataset/NP10154-130r.jpg  
 extracting: data/obj/Dataset/NP10154-130r.txt  
  inflating: data/obj/Dataset/NP10304-132r.jpg  
 extracting: data/obj/Dataset/NP10304-132r.txt  
  inflating: data/obj/Dataset/NP1034-9r.jpg  
 extracting: data/obj/Dataset/NP1034-9r.txt  
  inflating: data/obj/Dataset/NP10536-27r.jpg  
 extracting: data/obj/Dataset/NP10536-27r.txt  
  inflating: data/obj/Dataset/NP10546-134r.jpg  
 extracting: data/obj/Dataset/NP10546-134r.txt  
  inflating: data/obj/Dataset/NP10566-136r.jpg  
 extracting: data/obj/Dataset/NP10566-136r.txt  
  inflating: data/obj/Dataset/NP10604-138r.jpg  
 extracting: data/obj/Dataset/NP10604-138r.txt  
  inflating: data/obj/Dataset/NP10623-139

**6) Create train.txt file**

In [17]:
import glob
images_list = glob.glob("data/obj/Dataset/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [18]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-14 23:21:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  5.51MB/s    in 22s     

2020-08-14 23:22:12 (7.15 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 